# Load CSVs to Postgres

Workflow: 6 

Goal: Load all source files, TMDB, Reelgood, IMDB to database.

Result: The 3 database _Load tables are populated.

In [1]:
import os, sys
import time
import pandas as PD

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [3]:
os.chdir(os.getenv('PWD'))
sys.path.insert(0, os.getenv('PWD'))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "settings.py")
import django
django.setup()

'app_proj.settings'

In [4]:
import app_proj.settings as ST
import movies.models.models as MD

## Read & Insert CSVs

In [5]:
MOVIEDB_FILE = os.path.join(ST.BASE_DIR, 'movies/data/moviedb_subsample.csv')
REELGOOD_FILE = os.path.join(ST.BASE_DIR, 'movies/data/reelgood_subsample.csv')
IMDB_FILE = os.path.join(ST.BASE_DIR, 'movies/data/imdb_subsample.csv')

In [6]:
#MD.Editor.DeleteTable('All')

In [7]:
# the movie DB

moviedb_ls = MD.Editor.CSVtoDict(MOVIEDB_FILE)
len(moviedb_ls)

319

In [8]:
#moviedb_ls[12]

In [9]:
MD.Editor.InsertDictToTable(moviedb_ls, 'MovieDB_Load')
MD.MovieDB_Load.objects.all().count()

319

In [10]:
# reelgood

reelgood_ls = MD.Editor.CSVtoDict(REELGOOD_FILE)
len(reelgood_ls)

325

In [11]:
#reelgood_ls[90]

In [12]:
MD.Editor.InsertDictToTable(reelgood_ls, 'Reelgood_Load')
MD.Reelgood_Load.objects.all().count()

325

In [13]:
# imdb

imdb_ls = MD.Editor.CSVtoDict(IMDB_FILE)
len(imdb_ls)

317

In [14]:
#imdb_ls[0]

In [15]:
MD.Editor.InsertDictToTable(imdb_ls, 'IMDB_Load')
MD.IMDB_Load.objects.all().count()

317

## Create Master Movie Table

In [16]:
master_ls = MD.Editor.RunMasterMovies()
MD.Editor.InsertDictToTable(master_ls, 'MasterMovie')
MD.MasterMovie.objects.all().count()

319

In [17]:
master_df = PD.DataFrame(master_ls)
#master_df.head()
#master_df.info()

In [18]:
history_ls = MD.Reporter.GetDataHistory()
history_df = PD.DataFrame(history_ls).set_index('Column')
history_df

KeyError: 'Union-All'

In [ ]:
# for development, return the merged dataframe

df = MD.Editor.RunMaster Movies()
df[150:152]

In [ ]:
df.info()

In [ ]:
df.loc[list(range(140,160)), ['Title_tmdb', 'Score_tmdb', 'Votes_tmdb', 'Score_imdb', 'Votes_imdb']]

In [ ]:
df['Rating_rlgd'].value_counts()

In [ ]:
tmdb_genre = []
rlgd_genre = []
imdb_genre = []
for idx, row in df.iterrows():
    tmdb = row['Genres_tmdb'] or ''
    rlgd = row['Genres_rlgd'] or ''
    imdb = row['Genres_imdb'] or ''
    
    tmdb_genre += tmdb.split(', ') if type(tmdb)==str else []
    rlgd_genre += rlgd.split(', ') if type(rlgd)==str else []
    imdb_genre += imdb.split(', ') if type(imdb)==str else []
    
    tmdb_ct = PD.Series(tmdb_genre).value_counts()
    rlgd_ct = PD.Series(rlgd_genre).value_counts()
    imdb_ct = PD.Series(imdb_genre).value_counts()
    
#tmdb_ct

In [ ]:
df.loc[[111, 136, 148, 269, 272, 19, 44, 47], ['Title_tmdb', 'Budget_tmdb', 'Budget_imdb']]

In [ ]:
round( ((6.8 * 293) + (5.8 * 6457))/(293+6457) , 2)

df.dropna(subset=['Votes_imdb'])['Votes_tmdb'].astype('int').sum()
df.dropna(subset=['Votes_imdb'])['Votes_imdb'].astype('int').sum()

In [ ]:
MD.Editor.GetMasterMovie(df.iloc[56])

In [ ]:
MD.Editor.GetMasterMovie(df.iloc[166])